### Instituto Tecnológico de Costa Rica (ITCR)
### Escuela de Computación
### Curso: Inteligencia Artificial
 
### Tercera tarea programada 2022-I

### Parte 2 - ejercicio 2


Estudiantes: Juan Ignacio Navarro Navarro

In [36]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
import numpy as np
import csv



torch.manual_seed(1)

In [38]:
# Funciones utilitarias

def max_values(x):
    # Retorna el valor máximo y en índice o la posición del valor en un vector x.
    # Parámetros: 
    #    x: vector con los datos. 
    # Salida: 
    #    out: valor 
    #    inds: índice
    out, inds = torch.max(x,dim=1)   
    return out, inds

# Preparación de los datos 
def prepare_sequence(seq, to_ix):
    # Retorna un tensor con los indices del diccionario para cada palabras en una oración.
    # Parámetros:
    #   seq: oración
    #   to_ix: diccionario de palabras.
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

In [40]:
# Loading data to csv
df = pd.read_csv(r'tripadvisor_hotel_reviews.csv')
# Lowercase columns
df.columns = df.columns.str.lower()

# Diccionario de palabras con su calificación

training_data = []
for i in df.index:
    elem_tuple = df["review"][i].split(), int(df["rating"][i])
    training_data.append(elem_tuple)

# Diccionario las palabras
word_to_ix = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)

tag_to_ix = {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5}


#Ejemplo de procesamiento de una oración
inputs = prepare_sequence(training_data[0][0], word_to_ix)
print(training_data[0][0])                          
print(inputs)


['nice', 'hotel', 'expensive', 'parking', 'got', 'good', 'deal', 'stay', 'hotel', 'anniversary,', 'arrived', 'late', 'evening', 'took', 'advice', 'previous', 'reviews', 'did', 'valet', 'parking,', 'check', 'quick', 'easy,', 'little', 'disappointed', 'non-existent', 'view', 'room', 'room', 'clean', 'nice', 'size,', 'bed', 'comfortable', 'woke', 'stiff', 'neck', 'high', 'pillows,', 'not', 'soundproof', 'like', 'heard', 'music', 'room', 'night', 'morning', 'loud', 'bangs', 'doors', 'opening', 'closing', 'hear', 'people', 'talking', 'hallway,', 'maybe', 'just', 'noisy', 'neighbors,', 'aveda', 'bath', 'products', 'nice,', 'did', 'not', 'goldfish', 'stay', 'nice', 'touch', 'taken', 'advantage', 'staying', 'longer,', 'location', 'great', 'walking', 'distance', 'shopping,', 'overall', 'nice', 'experience', 'having', 'pay', '40', 'parking', 'night,']
tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  1,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 26, 27,  0, 28, 29

In [41]:
# Definición del modelo

# El modelo es una clase que debe heredar de nn.Module
class LSTMTagger(nn.Module):
    
    # Incialización del modelo
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim
 

        # Primero se pasa la entrada a través de una capa Embedding. 
        # Esta capa construye una representación de los tokens de 
        # un texto donde las palabras que tienen el mismo significado 
        # tienen una representación similar.
        
        # Esta capa captura mejor el contexto y son espacialmente 
        # más eficientes que las representaciones vectoriales (one-hot vector).
        # En Pytorch, se usa el módulo nn.Embedding para crear esta capa, 
        # que toma el tamaño del vocabulario y la longitud deseada del vector 
        # de palabras como entrada. 
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # El LSTM toma word_embeddings como entrada y genera los estados ocultos
        # con dimensionalidad hidden_dim.  
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # La capa lineal mapea el espacio de estado oculto 
        # al espacio de clases
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        # Pasada hacia adelante de la red. 
        # Parámetros:
        #    sentence: la oración a procesar
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))

        # Se utiliza softmax para devolver la probabilidad de cada etiqueta
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores


In [42]:
# Instanciación del modelo, definición de la función de pérdida y del optimizador   

# Hiperparámetros de la red
# Valores generalmente altos (32 o 64 dimensiones).
# Se definen pequeños, para ver cómo cambian los pesos durante el entrenamiento.

EMBEDDING_DIM = 6
HIDDEN_DIM = 6

# Instancia del modelo
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))

# Función de pérdida: Negative Log Likelihood Loss (NLLL). 
# Generalmente utilizada en problemas de clasificacion con C clases.
loss_function = nn.NLLLoss()

# Optimizador Stochastic Gradient Descent  
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [43]:
# Entrenamiento del modelo 

# Valores antes de entrenar
# El elemento i, j de la salida es la puntuación entre la etiqueta j para la palabra i.
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    
    print(training_data[0][0])
    
    # Clasificación    
    print(tag_scores)

# Épocas de entrenamiento
for epoch in range(500):  
    for sentence, tags in training_data:
        ## Paso 1. Pytorch acumula los gradientes.
        # Es necesario limpiarlos
        model.zero_grad()

        # Paso 2. Se preparan las entradas, es decir, se convierten a
        # tensores de índices de palabras.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # Paso 3. Se genera la predicción (forward pass).
        tag_scores = model(sentence_in)

        # Paso 4. se calcula la pérdida, los gradientes, y se actualizan los 
        # parámetros por medio del optimizador.
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

# Despligue de la puntuación luego del entrenamiento
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
   
    print("Resultados luego del entrenamiento para la primera frase")
    # Las palabras en una oración se pueden etiquetar de tres formas.
    # La primera oración tiene 4 palabras "El perro come manzana"
    # por eso el tensor de salida tiene 4 elementos. 
    # Cada elemento es un vector de pesos que indica cuál etiqueta tiene más
    # posibilidad de estar asociada a la palabra. Es decir hay que calcular 
    # la posición del valor máximo
    print(tag_scores)

['nice', 'hotel', 'expensive', 'parking', 'got', 'good', 'deal', 'stay', 'hotel', 'anniversary,', 'arrived', 'late', 'evening', 'took', 'advice', 'previous', 'reviews', 'did', 'valet', 'parking,', 'check', 'quick', 'easy,', 'little', 'disappointed', 'non-existent', 'view', 'room', 'room', 'clean', 'nice', 'size,', 'bed', 'comfortable', 'woke', 'stiff', 'neck', 'high', 'pillows,', 'not', 'soundproof', 'like', 'heard', 'music', 'room', 'night', 'morning', 'loud', 'bangs', 'doors', 'opening', 'closing', 'hear', 'people', 'talking', 'hallway,', 'maybe', 'just', 'noisy', 'neighbors,', 'aveda', 'bath', 'products', 'nice,', 'did', 'not', 'goldfish', 'stay', 'nice', 'touch', 'taken', 'advantage', 'staying', 'longer,', 'location', 'great', 'walking', 'distance', 'shopping,', 'overall', 'nice', 'experience', 'having', 'pay', '40', 'parking', 'night,']
tensor([[-2.0096, -1.8258, -1.9736, -1.5528, -1.9723, -1.5365],
        [-2.0170, -1.8243, -2.0222, -1.5407, -1.9466, -1.5313],
        [-2.0195, 

TypeError: 'int' object is not iterable